# Quantum Process Tomography via Neural Networks
## Complex polarization transformations
___

This notebook performs the process tomography for the experiments proposed in our paper 'Retrieving unitary polarization transformations via optimized quantum tomography'.

You can use this code for your experiments as well, by modelling your data in a similar way to the data available in the repository. 

________

First, the required libraries are imported:

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GaussianDropout
from tensorflow.keras import optimizers
import tensorflow as tf

By running the following cell, you import the experimental data.
Please select a process from 1 to 3, to replicate the corresponding experiment in the paper.

In [ ]:
num_pix=73 # specify the number of pixels num_pix*num_pix of your grid

process = "3"
path = "experimental_data/"
LL = np.loadtxt(path + process + "/LL.txt", dtype="f", delimiter="\t")
HH = np.loadtxt(path + process + "/HH.txt", dtype="f", delimiter="\t")
HL = np.loadtxt(path + process + "/HL.txt", dtype="f", delimiter="\t")
LD = np.loadtxt(path + process + "/LD.txt", dtype="f", delimiter="\t")
LH = np.loadtxt(path + process + "/LH.txt", dtype="f", delimiter="\t")
HD = np.loadtxt(path + process + "/HD.txt", dtype="f", delimiter="\t")

#the order must be LL, LH, LD, HL, HH, HD

mat_data=np.zeros([num_pix,num_pix,6])
mat_data[:,:,0]=LL
mat_data[:,:,1]=LH
mat_data[:,:,2]=LD
mat_data[:,:,3]=HL
mat_data[:,:,4]=HH
mat_data[:,:,5]=HD

If available, you can import the corresponding theoretical parameters to compute the fidelity of the reconstructed process.

In [ ]:
#select correct path
theta_th_mat=np.loadtxt(path + process +"/thetath.txt", dtype='f', delimiter='\t')
nx_th_mat=np.loadtxt(path + process +"/nxth.txt", dtype='f', delimiter='\t')
ny_th_mat=np.loadtxt(path + process +"/nyth.txt", dtype='f', delimiter='\t')
nz_th_mat=np.loadtxt(path + process +"/nzth.txt", dtype='f', delimiter='\t')

The function `compute_unitary` is used to compute the unitary $U$, given the parameters $\Theta\in[0,\pi]$ and $\mathbf{n}=(n_x,n_y,n_z)$ according to:

\begin{equation}
U=\begin{pmatrix}
\cos \Theta -i \sin \Theta \,n_z && -i\sin \Theta \,(n_x-i n_y)\\
-i\sin \Theta \,(n_x+i n_y) && \cos \Theta + i \sin \Theta \,n_z
\end{pmatrix}
\end{equation}

In [ ]:
def compute_unitary(Theta, nx, ny, nz):
    I = np.array([[1, 0], [0, 1]])
    sx = np.matrix([[0, 1], [1, 0]])
    sy = np.matrix([[0, -1j], [1j, 0]])
    sz = np.matrix([[1, 0], [0, -1]])
    return np.cos(Theta) * I - 1j * np.sin(Theta) * (nx * sx + ny * sy + nz * sz)

The function `fidelity` is used to compute the function used to measure the "distance" between the reconstructed and theoretical unitaries:

\begin{equation}
F=\frac{1}{2}\,\biggl|Tr(U_\text{th}^{\dagger}U_\text{exp})\biggr|
\end{equation}

In [ ]:
def fidelity(mat1,mat2):
    prod=np.trace(np.dot(np.conjugate(mat1.T),mat2))
    
    return 0.5*np.abs(prod)

We import the network trained with 6 input measurements $[LL, LH, LD, HL, HH, HD]$

In [ ]:
# load json and create model

json_file = open(r'./models/NN_6in.json', 'r') #path of NN json file

loaded_model_json = json_file.read()

json_file.close()

loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# load weights into new model

loaded_model.load_weights(r'./models/NN_6in.h5') #path of NN h5 file

print("Loaded model from disk")

## Pixel-by-pixel reconstruction

In [ ]:
flat_input=mat_data.reshape((num_pix*num_pix,6))

y_pred=loaded_model.predict(flat_input)
thetavect=y_pred[:,0]*np.pi
nx_vect=y_pred[:,1]*2 -1 
ny_vect=(y_pred[:,2]*2 -1)*np.sqrt(1-nx_vect**2)

nz_vect=np.sqrt(abs(1-nx_vect**2-ny_vect**2))

## Continuity and error correction
### First run

Select a maximum discontinuity in fidelity between close pixels (correction by parameters continuity)

In [ ]:
#minimum fidelity between neighbouring pixels for the 1st run
tol=0.9

#preliminary: continuity of the parameters between pixels 1 and 2

diff1=np.amax(np.array([abs(thetavect[1]-thetavect[0]),abs(nx_vect[1]-nx_vect[0]),abs(ny_vect[1]-ny_vect[0])]))
alt_vect=np.zeros([4])
alt_vect[0]=np.pi-thetavect[1]
alt_vect[1]=-nx_vect[1]
alt_vect[2]=-ny_vect[1]
alt_vect[3]=-nz_vect[1]
diff_alt_1=np.amax(np.array([abs(alt_vect[0]-thetavect[0]),abs(alt_vect[1]-nx_vect[0]),abs(alt_vect[2]-ny_vect[0])]))

if diff_alt_1<diff1:
    thetavect[1]=alt_vect[0]
    nx_vect[1]=alt_vect[1]
    ny_vect[1]=alt_vect[2]
    nz_vect[1]=alt_vect[3]
    
#imposing continuity of the parameters to choose between U and -U

for i in range(2,num_pix**2):
    diff1=np.amax(np.array([abs(thetavect[i]-thetavect[i-1]),abs(nx_vect[i]-nx_vect[i-1]),abs(ny_vect[i]-ny_vect[i-1])]))
    diff2=np.amax(np.array([abs(thetavect[i]-thetavect[i-2]),abs(nx_vect[i]-nx_vect[i-2]),abs(ny_vect[i]-ny_vect[i-2])]))
    alt_vect=np.zeros([4])
    alt_vect[0]=np.pi-thetavect[i]
    alt_vect[1]=-nx_vect[i]
    alt_vect[2]=-ny_vect[i]
    alt_vect[3]=-nz_vect[i]
    diff_alt_1=np.amax(np.array([abs(alt_vect[0]-thetavect[i-1]),abs(alt_vect[1]-nx_vect[i-1]),abs(alt_vect[2]-ny_vect[i-1])]))
    diff_alt_2=np.amax(np.array([abs(alt_vect[0]-thetavect[i-2]),abs(alt_vect[1]-nx_vect[i-2]),abs(alt_vect[2]-ny_vect[i-2])]))
    if min(diff_alt_1,diff_alt_2)<min(diff1,diff2):
        thetavect[i]=alt_vect[0]
        nx_vect[i]=alt_vect[1]
        ny_vect[i]=alt_vect[2]
        nz_vect[i]=alt_vect[3]
        
mat_theta=thetavect.reshape((num_pix,num_pix))
mat_nx=nx_vect.reshape((num_pix,num_pix))
mat_ny=ny_vect.reshape((num_pix,num_pix))
mat_nz=nz_vect.reshape((num_pix,num_pix))

def close_fid(i,j):
    
    fids=np.zeros([8])
    
    fids[0]=fidelity(compute_unitary(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),compute_unitary(mat_theta[(i-1)%num_pix,j%num_pix],mat_nx[(i-1)%num_pix,j%num_pix],mat_ny[(i-1)%num_pix,j%num_pix],mat_nz[(i-1)%num_pix,j%num_pix]))
    fids[1]=fidelity(compute_unitary(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),compute_unitary(mat_theta[(i-1)%num_pix,(j-1)%num_pix],mat_nx[(i-1)%num_pix,(j-1)%num_pix],mat_ny[(i-1)%num_pix,(j-1)%num_pix],mat_nz[(i-1)%num_pix,(j-1)%num_pix]))
    fids[2]=fidelity(compute_unitary(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),compute_unitary(mat_theta[i%num_pix,(j-1)%num_pix],mat_nx[i%num_pix,(j-1)%num_pix],mat_ny[i%num_pix,(j-1)%num_pix],mat_nz[i%num_pix,(j-1)%num_pix]))
    fids[3]=fidelity(compute_unitary(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),compute_unitary(mat_theta[(i+1)%num_pix,(j-1)%num_pix],mat_nx[(i+1)%num_pix,(j-1)%num_pix],mat_ny[(i+1)%num_pix,(j-1)%num_pix],mat_nz[(i+1)%num_pix,(j-1)%num_pix]))
    fids[4]=fidelity(compute_unitary(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),compute_unitary(mat_theta[(i+1)%num_pix,j%num_pix],mat_nx[(i+1)%num_pix,j%num_pix],mat_ny[(i+1)%num_pix,j%num_pix],mat_nz[(i+1)%num_pix,j%num_pix]))
    fids[5]=fidelity(compute_unitary(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),compute_unitary(mat_theta[(i+1)%num_pix,(j+1)%num_pix],mat_nx[(i+1)%num_pix,(j+1)%num_pix],mat_ny[(i+1)%num_pix,(j+1)%num_pix],mat_nz[(i+1)%num_pix,(j+1)%num_pix]))
    fids[6]=fidelity(compute_unitary(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),compute_unitary(mat_theta[i%num_pix,(j+1)%num_pix],mat_nx[i%num_pix,(j+1)%num_pix],mat_ny[i%num_pix,(j+1)%num_pix],mat_nz[i%num_pix,(j+1)%num_pix]))
    fids[7]=fidelity(compute_unitary(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),compute_unitary(mat_theta[(i-1)%num_pix,(j+1)%num_pix],mat_nx[(i-1)%num_pix,(j+1)%num_pix],mat_ny[(i-1)%num_pix,(j+1)%num_pix],mat_nz[(i-1)%num_pix,(j+1)%num_pix]))
    
    fid_val=max(fids)
    
    return fid_val

def inter_NN(i,j,mat_par):
    
    pars=np.zeros([8])
    trust=np.zeros([8])
    
    trust[0]=(close_fid(i-1,j)>tol)
    trust[1]=(close_fid(i-1,j-1)>tol)
    trust[2]=(close_fid(i,j-1)>tol)
    trust[3]=(close_fid(i+1,j-1)>tol)
    trust[4]=(close_fid(i+1,j)>tol)
    trust[5]=(close_fid(i+1,j+1)>tol)
    trust[6]=(close_fid(i,j+1)>tol)
    trust[7]=(close_fid(i-1,j+1)>tol)
    
    pars[0]=mat_par[(i-1)%num_pix,j]
    pars[1]=mat_par[(i-1)%num_pix,(j-1)%num_pix]
    pars[2]=mat_par[i,(j-1)%num_pix]
    pars[3]=mat_par[(i+1)%num_pix,(j-1)%num_pix]
    pars[4]=mat_par[(i+1)%num_pix,j]
    pars[5]=mat_par[(i+1)%num_pix,(j+1)%num_pix]
    pars[6]=mat_par[i,(j+1)%num_pix]
    pars[7]=mat_par[(i-1)%num_pix,(j+1)%num_pix]
    
    mean_par=sum(pars*trust)/sum(trust)
    
    return mean_par

#correcting wrong pixels by interpolating between their neighbours

for i in range(num_pix):
    for j in range(num_pix):
    
        if close_fid(i,j)<tol:
            mat_theta[i,j]=inter_NN(i,j,mat_theta)
            mat_nx[i,j]=inter_NN(i,j,mat_nx)
            mat_ny[i,j]=inter_NN(i,j,mat_ny)
            mat_nz[i,j]=inter_NN(i,j,mat_nz)
            
            norm=np.sqrt(mat_nx[i,j]**2 + mat_ny[i,j]**2 + mat_nz[i,j]**2)
            
            mat_nx[i,j]=mat_nx[i,j]/norm
            mat_ny[i,j]=mat_ny[i,j]/norm
            mat_nz[i,j]=mat_nz[i,j]/norm
            
alt_mat_theta=np.ones([num_pix,num_pix])*np.pi - mat_theta
alt_mat_nx=-mat_nx
alt_mat_ny=-mat_ny
alt_mat_nz=-mat_nz

def max_par_jump(i,j,k,h):
    
    return np.amax(np.array([abs(mat_theta[i,j]-mat_theta[i-1,j-1])/np.pi,abs(mat_nx[i,j]-mat_nx[i-1,j-1])/2,abs(mat_ny[i,j]-mat_ny[i-1,j-1])/2]))

def alt_max_par_jump(i,j,k,h):
    
    return np.amax(np.array([abs(alt_mat_theta[i,j]-mat_theta[i-1,j-1])/np.pi,abs(alt_mat_nx[i,j]-mat_nx[i-1,j-1])/2,abs(alt_mat_ny[i,j]-mat_ny[i-1,j-1])/2]))

def close_jump(i,j):
    
    jumps=np.zeros([8])
    
    jumps[0]=max_par_jump(i,j,(i-1)%num_pix,j)
    jumps[1]=max_par_jump(i,j,(i-1)%num_pix,(j-1)%num_pix)
    jumps[2]=max_par_jump(i,j,i,(j-1)%num_pix)
    jumps[3]=max_par_jump(i,j,(i+1)%num_pix,(j-1)%num_pix)
    jumps[4]=max_par_jump(i,j,(i+1)%num_pix,j)
    jumps[5]=max_par_jump(i,j,(i+1)%num_pix,(j+1)%num_pix)
    jumps[6]=max_par_jump(i,j,i,(j+1)%num_pix)
    jumps[7]=max_par_jump(i,j,(i-1)%num_pix,(j+1)%num_pix)
    
    jump_val=min(jumps)
    
    return jump_val

def alt_close_jump(i,j):
    
    jumps=np.zeros([8])
    
    jumps[0]=alt_max_par_jump(i,j,(i-1)%num_pix,j)
    jumps[1]=alt_max_par_jump(i,j,(i-1)%num_pix,(j-1)%num_pix)
    jumps[2]=alt_max_par_jump(i,j,i,(j-1)%num_pix)
    jumps[3]=alt_max_par_jump(i,j,(i+1)%num_pix,(j-1)%num_pix)
    jumps[4]=alt_max_par_jump(i,j,(i+1)%num_pix,j)
    jumps[5]=alt_max_par_jump(i,j,(i+1)%num_pix,(j+1)%num_pix)
    jumps[6]=alt_max_par_jump(i,j,i,(j+1)%num_pix)
    jumps[7]=alt_max_par_jump(i,j,(i-1)%num_pix,(j+1)%num_pix)
    
    jump_val=min(jumps)
    
    return jump_val

#NN continuity correction (now in 2D)

for i in range(num_pix):
    for j in range(num_pix):
    
        if alt_close_jump(i,j)<close_jump(i,j):
            mat_theta[i,j]=alt_mat_theta[i,j]
            mat_nx[i,j]=alt_mat_nx[i,j]
            mat_ny[i,j]=alt_mat_ny[i,j]
            mat_nz[i,j]=alt_mat_nz[i,j]
            
            norm=np.sqrt(mat_nx[i,j]**2 + mat_ny[i,j]**2 + mat_nz[i,j]**2)
            
            mat_nx[i,j]=mat_nx[i,j]/norm
            mat_ny[i,j]=mat_ny[i,j]/norm
            mat_nz[i,j]=mat_nz[i,j]/norm

### Second run

Select a maximum discontinuity in fidelity between close pixels (correction by parameters continuity)

In [ ]:
#minimum fidelity between neighbouring pixels for the 2nd run
tol2=0.95

thetavect2=mat_theta.reshape((num_pix*num_pix))
nx_vect2=mat_nx.reshape((num_pix*num_pix))
ny_vect2=mat_ny.reshape((num_pix*num_pix))
nz_vect2=mat_nz.reshape((num_pix*num_pix))

#imposing continuity of the parameters to choose between U and -U (again)

for i in range(2,num_pix**2):
    diff1=np.amax(np.array([abs(thetavect2[i]-thetavect2[i-1]),abs(nx_vect2[i]-nx_vect2[i-1]),abs(ny_vect2[i]-ny_vect2[i-1])]))
    diff2=np.amax(np.array([abs(thetavect2[i]-thetavect2[i-2]),abs(nx_vect2[i]-nx_vect2[i-2]),abs(ny_vect2[i]-ny_vect2[i-2])]))
    alt_vect2=np.zeros([4])
    alt_vect2[0]=np.pi-thetavect2[i]
    alt_vect2[1]=-nx_vect2[i]
    alt_vect2[2]=-ny_vect2[i]
    alt_vect2[3]=-nz_vect2[i]
    diff_alt_1=np.amax(np.array([abs(alt_vect2[0]-thetavect2[i-1]),abs(alt_vect2[1]-nx_vect2[i-1]),abs(alt_vect2[2]-ny_vect2[i-1])]))
    diff_alt_2=np.amax(np.array([abs(alt_vect2[0]-thetavect2[i-2]),abs(alt_vect2[1]-nx_vect2[i-2]),abs(alt_vect2[2]-ny_vect2[i-2])]))
    if min(diff_alt_1,diff_alt_2)<min(diff1,diff2):
        thetavect2[i]=alt_vect2[0]
        nx_vect2[i]=alt_vect2[1]
        ny_vect2[i]=alt_vect2[2]
        nz_vect2[i]=alt_vect2[3]
        
mat_theta=thetavect2.reshape((num_pix,num_pix))
mat_nx=nx_vect2.reshape((num_pix,num_pix))
mat_ny=ny_vect2.reshape((num_pix,num_pix))
mat_nz=nz_vect2.reshape((num_pix,num_pix))

#correcting wrong pixels by interpolating between their neighbours (again)

for i in range(num_pix):
    for j in range(num_pix):
    
        if close_fid(i,j)<tol2:
            mat_theta[i,j]=inter_NN(i,j,mat_theta)
            mat_nx[i,j]=inter_NN(i,j,mat_nx)
            mat_ny[i,j]=inter_NN(i,j,mat_ny)
            mat_nz[i,j]=inter_NN(i,j,mat_nz)
            
alt_mat_theta=np.ones([num_pix,num_pix])*np.pi - mat_theta
alt_mat_nx=-mat_nx
alt_mat_ny=-mat_ny
alt_mat_nz=-mat_nz

for j in range(num_pix):
    for i in range(num_pix):
    
        if alt_close_jump(-i,-j)<close_jump(-i,-j):
            mat_theta[-i,-j]=alt_mat_theta[-i,-j]
            mat_nx[-i,-j]=alt_mat_nx[-i,-j]
            mat_ny[-i,-j]=alt_mat_ny[-i,-j]
            mat_nz[-i,-j]=alt_mat_nz[-i,-j]

In [ ]:
#if total inversion of the map phase is needed

mat_theta=np.ones([num_pix,num_pix])*np.pi - mat_theta
mat_nx=-mat_nx
mat_ny=-mat_ny
mat_nz=-mat_nz

## Parameters plots (reconstructed and theoretical)

In [ ]:
xvals=np.arange(0,num_pix)
yvals=np.arange(0,num_pix)
y,x=np.meshgrid(xvals,yvals)

$\Theta$:

In [ ]:
plt.figure(1)
fig, ax = plt.subplots(figsize=(12, 8))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    mat_theta, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=-1, vmax=1
)
ax.set_xlabel(r"$x$", fontsize=14)
ax.set_ylabel(r"$y$", fontsize=14)
cbar = ax.collections[0].colorbar
cbar.set_ticks([-1, 1])
cbar.set_ticklabels(["-1", "1"])
plt.tight_layout()

plt.figure(1)
fig, ax = plt.subplots(figsize=(12, 8))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    theta_th_mat, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=-1, vmax=1
)
ax.set_xlabel(r"$x$", fontsize=14)
ax.set_ylabel(r"$y$", fontsize=14)
cbar = ax.collections[0].colorbar
cbar.set_ticks([-1, 1])
cbar.set_ticklabels(["-1", "1"])
plt.tight_layout()

$n_x$:

In [ ]:
plt.figure(1)
fig, ax = plt.subplots(figsize=(12, 8))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    mat_nx, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=-1, vmax=1
)
ax.set_xlabel(r"$x$", fontsize=14)
ax.set_ylabel(r"$y$", fontsize=14)
cbar = ax.collections[0].colorbar
cbar.set_ticks([-1, 1])
cbar.set_ticklabels(["-1", "1"])
plt.tight_layout()

plt.figure(1)
fig, ax = plt.subplots(figsize=(12, 8))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    nx_th_mat, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=-1, vmax=1
)
ax.set_xlabel(r"$x$", fontsize=14)
ax.set_ylabel(r"$y$", fontsize=14)
cbar = ax.collections[0].colorbar
cbar.set_ticks([-1, 1])
cbar.set_ticklabels(["-1", "1"])
plt.tight_layout()

$n_y$:

In [ ]:
plt.figure(1)
fig, ax = plt.subplots(figsize=(12, 8))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    mat_ny, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=-1, vmax=1
)
ax.set_xlabel(r"$x$", fontsize=14)
ax.set_ylabel(r"$y$", fontsize=14)
cbar = ax.collections[0].colorbar
cbar.set_ticks([-1, 1])
cbar.set_ticklabels(["-1", "1"])
plt.tight_layout()

plt.figure(1)
fig, ax = plt.subplots(figsize=(12, 8))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    ny_th_mat, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=-1, vmax=1
)
ax.set_xlabel(r"$x$", fontsize=14)
ax.set_ylabel(r"$y$", fontsize=14)
cbar = ax.collections[0].colorbar
cbar.set_ticks([-1, 1])
cbar.set_ticklabels(["-1", "1"])
plt.tight_layout()

$n_z$:

In [ ]:
plt.figure(1)
fig, ax = plt.subplots(figsize=(12, 8))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    mat_nz, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=-1, vmax=1
)
ax.set_xlabel(r"$x$", fontsize=14)
ax.set_ylabel(r"$y$", fontsize=14)
cbar = ax.collections[0].colorbar
cbar.set_ticks([-1, 1])
cbar.set_ticklabels(["-1", "1"])
plt.tight_layout()

plt.figure(1)
fig, ax = plt.subplots(figsize=(12, 8))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    nz_th_mat, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=-1, vmax=1
)
ax.set_xlabel(r"$x$", fontsize=14)
ax.set_ylabel(r"$y$", fontsize=14)
cbar = ax.collections[0].colorbar
cbar.set_ticks([-1, 1])
cbar.set_ticklabels(["-1", "1"])
plt.tight_layout()

The pixel-by-pixel fidelities are computed:

In [ ]:
def op_par(En,nx,ny,nz):
    mat=np.zeros([2,2],dtype=complex)
    
    mat[0,0]=np.cos(En) - 1j*np.sin(En)*nz
    mat[0,1]=-1j*np.sin(En)*(nx - 1j*ny)
    mat[1,0]=-1j*np.sin(En)*(nx + 1j*ny)
    mat[1,1]=np.cos(En) + 1j*np.sin(En)*nz
    
    return mat

Fvals=np.zeros([num_pix,num_pix])
for i in range(num_pix):
    for j in range(num_pix):
        netU=op_par(mat_theta[i,j],mat_nx[i,j],mat_ny[i,j],mat_nz[i,j])
        thU=op_par(theta_th_mat[i,j],nx_th_mat[i,j],ny_th_mat[i,j],nz_th_mat[i,j])
        Fvals[i,j]=fidelity(netU,thU)

The mean fidelity is computed:

In [ ]:
meanF=np.mean(Fvals)
meanF